# Introduction



In [1]:
import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress
client = Client('localhost:8786')
client

print d

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [762]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
from dask import delayed
import dask.array as da
import math
from matplotlib import pyplot as plt
import seaborn as sns
import hdbscan 
from kmodes.kmodes import KModes
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
spotify = dd.read_csv('/Volumes/T_Drive/Data/log_0_20180715_000000000000.csv', blocksize="10MB")

In [ ]:
spotify.info()

In [ ]:
spotify.head()

In [ ]:
# see tail of this particular chunk
spotify.tail()

In [ ]:
spotify.dtypes

In [ ]:
spotify.shape[0].compute()

In [ ]:
print(spotify.isnull().sum().compute())

In [ ]:
# How many sessions are there?
spotify.session_id.unique().compute()

In [ ]:
# Make booleans numeric

def make_bool_num(df):
    for col in df:
        if df[col].dtype == 'bool':
            df[col] = df[col] * 1
    return df[col] 

make_bool_num(spotify).compute()

In [ ]:
spotify.dtypes

session_id - unique identifier for the session that this row is a part of

session_position {1-20} - position of row within session

session_length {10-20} - number of rows in session

track_id_clean - unique identifier for the track played. This is linked with track_id in the track features and metadata table.

skip_1 - Boolean indicating if the track was only played very briefly

skip_2 - Boolean indicating if the track was only played briefly

skip_3 - Boolean indicating if most of the track was played

not_skipped Boolean indicating that the track was played in its entirety

context_switch - Boolean indicating if the user changed context between the previous row and the current row. This could for example occur if the user switched from one playlist to another.

no_pause_before_play - Boolean indicating if there was no pause between playback of the previous track and this track

short_pause_before_play - Boolean indicating if there was a short pause between playback of the previous track and this track

long_pause_before_play - Boolean indicating if there was a long pause between playback of the previous track and this track

hist_user_behavior_n_seekfwd - Number of times the user did a seek forward within track

hist_user_behavior_n_seekback - Number of times the user did a seek back within track

hist_user_behavior_is_shuffle - Boolean indicating if the user encountered this track while shuffle mode was activated

hour_of_day {0-23} - The hour of day

date - E.g. 2018-09-18 - The date

premium - Boolean indicating if the user was on premium or not. This has potential implications for skipping behavior.

context_type - E.g. editorial playlist - what type of context the playback occurred within

hist_user_behavior_reason_start - E.g. fwdbtn - the user action which led to the current track being played

hist_user_behavior_reason_end - E.g. trackdone - the user action which led to the current track playback ending

In [ ]:
# Make a slice of ints for histograms
spotify_int = spotify.select_dtypes(include='int64')

In [ ]:
spotify_int.info()

In [ ]:
spotify_int.head()

In [ ]:
def get_dist(df, figsize, rotation, hspace, wspace):
    plt.figure(figsize=(figsize))
    for i, col in enumerate(df):
        num_vals = len(df.columns)
        num_cols = math.ceil(num_vals / 5)
        num_rows = math.ceil(num_vals / num_cols)
        plt.subplot(num_rows, num_cols, i+1)
        plt.hist(df[col], bins=20)
        plt.title('Histogram of {}'.format(col))
        plt.xlabel('{}'.format(col))
        plt.ylabel('Observations')
        plt.xticks(rotation=rotation)
        plt.subplots_adjust(hspace=hspace, wspace=wspace)

get_dist(spotify_int.compute(), (22,15), 'horizontal', .5, .5)

Okay, so we definitely have at least 10 Bernoulli distributions because of the booleans. I want to look at hist_user_behavior_n_seekfwd and hist_user_behavior_n_seekback to see exactly what values they contain.

In [ ]:
spotify['hist_user_behavior_n_seekfwd'].value_counts().compute()

In [ ]:
spotify['hist_user_behavior_n_seekback'].value_counts().compute()

Alright, so there's actually quite a variety of values in these columns (definitely some outliers as well). Of course, most people didn't seek backwards or forwards within a track at all, but we can see that a few people did many times.

Now we'll want to see the distribution for the non-numeric variables (objects) in the user interaction df. Let's look at which ones they are first.

In [ ]:
spotify_obj = spotify.select_dtypes(include='object')

In [ ]:
spotify_obj.head()

In [ ]:
# Find number of unique dates in the dataset 
pd.set_option("display.max_rows", 70)
print(spotify.date.unique().compute())

In [ ]:
# I want to pull out just the month & day from date, since that could be interesting & most years are 2018

# First convert date from object to datetime
spotify['date'] = dd.to_datetime(spotify['date'])

# Extract just the day of the month from date
spotify['day'] = spotify.date.dt.day

# Check all represented days are still there
print(spotify['day'].unique().compute())

In [ ]:
# Now we'll get month
spotify['month'] = spotify.date.dt.month
 
# Check both months are there
print(spotify['month'].unique().compute())

In [ ]:
# Now I need to put month & day together in one int variable, since we need them both together, as some days overlap

# Make a month_day column that includes both day and month as float
spotify['month_day'] = spotify['month'] + (spotify['day'] / 100)

In [ ]:
# Check this was done correctly
spotify.head()

In [ ]:
print(spotify.month_day.unique().compute())

In [ ]:
# Now, since there are a few different years here, we'll pull those out separately too just in case that's important

# Get year variable
spotify['year'] = spotify.date.dt.year

# Check both months are there
print(spotify['year'].unique().compute())

In [ ]:
# Lets also check the value_counts for 'year,' since I think 2018 is way more represented 
print(spotify.year.value_counts().unique().compute())

In [ ]:
spotify.head()

In [ ]:
# Now we need to see what the unique values of the other objects are
print(spotify.context_type.value_counts().compute())                       
print(spotify.hist_user_behavior_reason_start.value_counts().compute())    
print(spotify.hist_user_behavior_reason_end.value_counts().compute())

In [ ]:
# Let's recode the relevant objects to make them numeric before running barplots & other EDA

# Produce a dictionary associating the alphabetized values of a col with their numeric indices 
def make_numeric(df_series):
    df_series_sorted = sorted(df_series.unique())
    num_dict = {}
    for index, val in enumerate(df_series_sorted):
        num_dict[val] = index    
    return num_dict

# Recode the col using the index numbers generated from the previous make_numeric function
def recode(df, column_name, recode_column_name, dtype):
    value_dict = make_numeric(df[column_name])
    df[recode_column_name] = df.apply(lambda row: value_dict[row[column_name]], axis=1, meta=dtype)
    print(value_dict)

In [ ]:
# Apply recode function to objects
recode(spotify, "context_type", "context_type_num", "int64")
recode(spotify, "hist_user_behavior_reason_start", "reason_start_num", "int64")
recode(spotify, "hist_user_behavior_reason_end", "reason_end_num", "int64")

In [ ]:
# Okay, now we're ready to make a slice without the id's & other objects so we can run barplots
spotify_eda = spotify.drop(columns=[
    'date', 
    'session_id', 
    'track_id_clean', 
    'context_type', 
    'hist_user_behavior_reason_start',
    'hist_user_behavior_reason_end'
])

In [ ]:
# Double-check all are ints and floats
spotify_eda.dtypes

In [ ]:
# Check 'not_skipped' to make sure it represents whether the user skipped the track at any point

print(spotify_eda.not_skipped.compute())
print(spotify_eda.skip_1.compute())
print(spotify_eda.skip_2.compute())
print(spotify_eda.skip_3.compute())

Great, so not_skipped does represent whether or not a song was skipped at any point during its play. 

However, I think the way not_skipped is recorded makes interpreting it a bit confusing, simply because 1 means 'not' skipped, which is a negative. I like to have 0s be negative, so I'm just going to create a variable - 'skipped' - that takes the value 1 if the song was skipped, and 0 if the song was not skipped.

The following is not the most elegant solution, but I wanted to use np.where, which I could not find an appropriate counterpart for in dask, so I converted the df to pandas, used np.where to make the 'skipped' column, and then converted the df back to a dask df.

In [ ]:
spotify_eda_pd = spotify_eda.compute()

spotify_eda_pd['skipped'] = np.where(spotify_eda_pd['not_skipped']>0, 0, 1)

In [ ]:
spotify_eda_pd.head()

Great, the first 5 observations in skipped are all 0s, whereas the first 5 in not_skipped are all 1s.

In [ ]:
# Convert back to dask df
spotify_eda = dd.from_pandas(spotify_eda_pd, npartitions=1)

In [ ]:
spotify_eda.info()     # Making sure it's a dask df & col number is correct

In [ ]:
# Show a countplot for all columns in eda dataset

def countplot(df, rotation): 
    plt.figure(figsize=(15,10))
    for col in df:
        sns.catplot(x=col, kind='count', palette='Greens_r', data=df)
        plt.title('Distribution of {}'.format(col))
        plt.xlabel('{}'.format(col))
        plt.ylabel('Count')
        plt.xticks(rotation=rotation)
        plt.show()       

In [ ]:
# # Use countplot function for our df
# countplot(spotify_eda.compute(), 'vertical')
# Alright we'll do this when we have the fucking cluster synced

In [ ]:
# Check what number of tracks in a listening session is most common
print(spotify_eda.session_length.value_counts().compute())

In [ ]:
# Make slice with only those listening sessions that have 20 songs in them
spotify_20 = spotify_eda[spotify_eda['session_length'] == 20]

spotify_20.info()

In [ ]:
# Check length with above
len(spotify_20)

In [ ]:
# Let's now limit ourselves to the 1st, middle, and last songs in each session
spotify20_3tracks =  spotify_20[(spotify_20[
    'session_position'] == 1) | (spotify_20[
    'session_position'] == 10) | (spotify_20[
    'session_position'] == 20)]

In [ ]:
spotify20_3tracks.info()

In [ ]:
spotify20_3tracks.head()

In [ ]:
len(spotify20_3tracks)

In [ ]:
# ~Split spotify20_3tracks into 3 slices~

# A slice with all of the 1st tracks in each listening session:
spotify_track1 = spotify20_3tracks[spotify20_3tracks['session_position'] == 1]

# A slice with all of the 10th tracks in each listening session:
spotify_track10 = spotify20_3tracks[spotify20_3tracks['session_position'] == 10]

# A slice with all of the last (20th) track in each listening session:
spotify_track20 = spotify20_3tracks[spotify20_3tracks['session_position'] == 20]

In [ ]:
len(spotify_track1)

In [ ]:
len(spotify_track10)

In [ ]:
len(spotify_track20)

Great, they're all equal, as expected.

First let's do some EDA for these slices, one at a time. Then perhaps we can compare the slices to each other, to tease out how they might diverge (if they do at all).

In [ ]:
# First, let's persist these datasets to memory, since they're smaller in scope than the original
spotify_track1 = client.persist(spotify_track1)
spotify_track10 = client.persist(spotify_track10)
spotify_track20 = client.persist(spotify_track20)

In [ ]:
countplot(spotify_track1.compute(), 'vertical')

In [ ]:
countplot(spotify_track10.compute(), 'vertical')

In [ ]:
countplot(spotify_track20.compute(), 'vertical')

Okay let's do a bivariate analysis with 'skipped' and the other variables in the dataset, for each of the 3 slices

In [ ]:
# Show a barplot for all columns in eda dataset with the given y (passed in as string)

def barplot(df, rotation, y): 
    for col in df:
        plt.figure(figsize=(15,10))
        sns.barplot(x=col, y=y, palette='Greens_r', data=df)
        plt.title('Relationship between {} & {}'.format(col, y))
        plt.xlabel('{}'.format(col))
        plt.ylabel('{}'.format(y))
        plt.xticks(rotation=rotation)
        plt.show()

In [ ]:
barplot(spotify_track1.compute(), 'vertical', 'skipped')

In [ ]:
barplot(spotify_track10.compute(), 'vertical', 'skipped')

In [ ]:
barplot(spotify_track20.compute(), 'vertical', 'skipped')

Okay, some boxplots would be fun - they'll give better measures of central tendency.

In [ ]:
# Show a boxplot for all columns in eda dataset with the given y (passed in as string) - in case I need to use this

def boxplot(df, rotation, y): 
    for col in df:
        plt.figure(figsize=(20,25))
        sns.boxplot(x=col, y=y, palette='Greens_r', data=df)
        plt.title('Relationship between {} & {}'.format(col, y))
        plt.xlabel('{}'.format(col))
        plt.ylabel('{}'.format(y))
        plt.xticks(rotation=rotation)
        plt.show()

In [ ]:
print(spotify_track1['premium'].name)

In [ ]:
# I think it would be good to compare the 3 slices to each other also for some vars; I'll make a function for that

def plot_side_by_side(series_list, y1, y2, y3, suptitle):
    plt.figure(figsize=(20,15))
    plt.suptitle('{}'.format(suptitle))
    
    plt.subplot(1, 3, 1)
    sns.barplot(x=series_list[0], y=y1, palette='Greens_r')

    plt.subplot(1, 3, 2)
    sns.barplot(x=series_list[1], y=y2, palette='Greens_r')

    
    plt.subplot(1, 3, 3)
    sns.barplot(x=series_list[2], y=y3, palette='Greens_r')  

In [ ]:
# Call function to look at relationship between premium & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['premium'].compute(), 
    spotify_track10['premium'].compute(), 
    spotify_track20['premium'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing premium & skipped for 1st, 10th, and 20th tracks'
)

In [ ]:
# Call function to look at relationship between context_type & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['context_type_num'].compute(), 
    spotify_track10['context_type_num'].compute(), 
    spotify_track20['context_type_num'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing context_type & skipped for 1st, 10th, and 20th tracks'
)

That's interesting - so for the 1st track in a listening session, it was most common for a track to be skipped if it was spotify radio; however, for both the 10th and 20th tracks in a listening session, it was most common for tracks to be skipped if they were in Spotify's top charts.

In [ ]:
# Call function to look at relationship between context_switch & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['context_switch'].compute(), 
    spotify_track10['context_switch'].compute(), 
    spotify_track20['context_switch'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing context_switch & skipped for 1st, 10th, and 20th tracks'
)

In [ ]:
# Call function to look at relationship between hist_user_behavior_is_shuffle & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['hist_user_behavior_is_shuffle'].compute(), 
    spotify_track10['hist_user_behavior_is_shuffle'].compute(), 
    spotify_track20['hist_user_behavior_is_shuffle'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing hist_user_behavior_is_shuffle & skipped for 1st, 10th, and 20th tracks'
)

More users skipped a song if they were in shuffle mode at the time.

In [ ]:
# Call function to look at relationship between month & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['month'].compute(), 
    spotify_track10['month'].compute(), 
    spotify_track20['month'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing month & skipped for 1st, 10th, and 20th tracks'
)

In [ ]:
# Call function to look at relationship between hour_of_day & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['hour_of_day'].compute(), 
    spotify_track10['hour_of_day'].compute(), 
    spotify_track20['hour_of_day'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing hour_of_day & skipped for 1st, 10th, and 20th tracks'
)

In [ ]:
# Call function to look at relationship between reason_start_num  & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['reason_start_num'].compute(), 
    spotify_track10['reason_start_num'].compute(), 
    spotify_track20['reason_start_num'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing reason_start_num & skipped for 1st, 10th, and 20th tracks'
)

In [ ]:
# Call function to look at relationship between reason_end_num & skipped across 3 slices
plot_side_by_side(
    [
    spotify_track1['reason_end_num'].compute(), 
    spotify_track10['reason_end_num'].compute(), 
    spotify_track20['reason_end_num'].compute()
], 
    spotify_track1['skipped'].compute(), 
    spotify_track10['skipped'].compute(), 
    spotify_track20['skipped'].compute(),
    'Comparing reason_end_num & skipped for 1st, 10th, and 20th tracks'
)

Well, that was interesting.

Now we need to do some feature engineering before getting into machine learning. The first thing I'm going to do is run spearman correlations & find out if any of the features are highly multicollinear with each other. To do this, I'm going to have to use the pandas version of the df, because dask is currently not supporting the spearman method of correlation, only pearson. I'm not going to run pearson correlations, because those are more for continuous data, and this data is not continuous.

In [ ]:
spotify_eda_pd.head()

In [ ]:
spotify_eda_pd.corr(method='spearman')

Great, now let's look at this in a heatmap

In [ ]:
# Assign correlation matrix to a variable
corrmat = spotify_eda_pd.corr(method='spearman')

# Set dimensions of heatmap
hm_dims = (30, 22)

# Create heatmap
fig, ax = plt.subplots(figsize=hm_dims)
sns.heatmap(corrmat, square=False, annot=True, linewidths=.4, ax=ax)
ax.set_ylim(len(corrmat)-0.5, -0.5)      #using this line b/c workaround for a bug that cuts off top & bottom
plt.show()

In [ ]:
# Alright, now a VIF to check the magnitude of the multicollinearity
def vif_cal(input_data, dependent_col):
    x_vars=input_data.drop([dependent_col], axis=1)
    xvar_names=x_vars.columns
    for i in range(0,xvar_names.shape[0]):
        y=x_vars[xvar_names[i]]
        x=x_vars[xvar_names.drop(xvar_names[i])]
        rsq=sm.OLS(endog=y, exog=x.astype(float), data=x_vars).fit().rsquared
        vif=round(1/(1-rsq),2)
        print (xvar_names[i], " VIF = " , vif)
        
#Call function, using sleptim1 as dependent var
vif_cal(input_data=spotify_eda_pd, dependent_col="skipped")

Okay this will help figure out which variables should be removed before clustering. I'll examine the variables that have a VIF > 10 (the threshold) one by one here, to make sure it makes sense to remove them, then I'll remove them, and I'll check to make sure all VIFs are now lower than 10. Then I'll remove these variables from each of the slices we're going to use.

**session_length** - This should be fine to remove, as all of the slices of the dataset are 20, so we will know what that is regardless. Ditto with **session_position** for each of the specific tracks' datasets

**skip_3** - I'll remove this, as well as **not_skipped**, because not_skipped would be redundant, as skipped is exactly the same thing.

**no_pause_before_play** - Keep this in for now; see if removing no_pause_before_play fixes the high VIF; however, I'm not quite so interested in the granularity offered by long_pause and short_pause

**long_pause_before play** - Fine with removing this

**day** - For some reason, this is infinity.. We'll remove it for sure. Also, I would have removed it regardless of VIF, since I don't need that level of granularity here

**month** - I'll try and keep month, because it will probably be the only real useful measure of time 

**month_day** - Same as day, too granular anyway

**year** - Remove - most are 2018 anyway, so it shouldn't really factor into the analysis

**reason_end_num** - It makes sense this would be highly multicollinear with reason_start_num; we'll remove it. 

In [ ]:
drop_list = [
    'session_length',
    'session_position',
    'skip_3', 
    'not_skipped',  
    'long_pause_before_play',
    'day',
    'month_day',
    'year',
    'reason_end_num'
]

spotify_eda_pd = spotify_eda_pd.drop(columns=drop_list)

In [ ]:
# Check out df with dropped stuff
spotify_eda_pd.info()

In [ ]:
# Re-run correlation matrix now, see how correlated vars are now
corrmat = spotify_eda_pd.corr(method='spearman')

# Set dimensions of heatmap
hm_dims = (30, 22)

# Create heatmap
fig, ax = plt.subplots(figsize=hm_dims)
sns.heatmap(corrmat, square=False, annot=True, linewidths=.4, ax=ax)
ax.set_ylim(len(corrmat)-0.5, -0.5)      #using this line b/c workaround for a bug that cuts off top & bottom
plt.show()

Okay, it's a lot better already. I can see that no_pause_before_play, and short_pause_before are of course still correlated, but we won't remove one until we check the VIF. Ditto with skipped and sip_1 & skip_2

In [ ]:
# Re-run VIF now to check magnitude of correlations
vif_cal(input_data=spotify_eda_pd, dependent_col="skipped")

Alright, looking good. I'd rather keep **no_pause_before_play** than **short_pause_before_play**. I believe no_pause's VIF will decrease once I remove short_pause. As to dealing with **month**, I'm going to go ahead and keep it despite the high VIF, because the correlation matrix/heatmap shows that it is not actually highly correlated with any of the variables that are left. 

In [ ]:
# Remove short_pause_before_play
spotify_eda_pd = spotify_eda_pd.drop(columns='short_pause_before_play')

In [ ]:
# Check correlations for what should be the final time
corrmat = spotify_eda_pd.corr(method='spearman')

# Set dimensions of heatmap
hm_dims = (30, 22)

# Create heatmap
fig, ax = plt.subplots(figsize=hm_dims)
sns.heatmap(corrmat, square=False, annot=True, linewidths=.4, ax=ax)
ax.set_ylim(len(corrmat)-0.5, -0.5)      #using this line b/c workaround for a bug that cuts off top & bottom
plt.show()

In [ ]:
# Now run VIF for what I hope to be the last time
vif_cal(input_data=spotify_eda_pd, dependent_col="skipped")

Hooray, we're all set now. I know the month's VIF looks scary, but again, it's not actually correlated with anything. I see no problem with keeping both of skip_1 and skip_2, because their VIFs are fine, despite their sort of high correlation.

Now it's time to drop all these variables from our 3 slices.

In [ ]:
# Set up a list of vars to drop
drop_list2 = [
    'session_length',
    'session_position',
    'skip_3', 
    'not_skipped',  
    'long_pause_before_play',
    'short_pause_before_play',
    'day',
    'month_day',
    'year',
    'reason_end_num'
]

# Drop them!
Xclusters_track1 = spotify_track1.drop(columns=drop_list2)
Xclusters_track10 = spotify_track10.drop(columns=drop_list2)
Xclusters_track20 = spotify_track20.drop(columns=drop_list2)

In [ ]:
# Check out a slice to see if it's all good
Xclusters_track1.info()

In [ ]:
Xclusters_track1.head()

In [ ]:
# Just to be extra double sure, I'm going to run VIF again, this time with a slice, to make sure it mirrors the df's
vif_cal(input_data=Xclusters_track1.compute(), dependent_col="skipped")

Well great, month's VIF is even lower here. I'd say we're all set.

Now we need to get dummies for all of these. Because dask is not great for converting to dummies (in my opinion at least), we are going to use the pandas version of the dataframe to get dummies.

However, before we get the dummies, we'll need to do a bit more feature engineering to make the dummies better suited to the clusters we'll create.

In [ ]:
# Make hist_user_behavior_n_seekfwd ordinal buckets so there's not quite as many unique values to deal with

# First, check their unique values in each of the slices
slice_list = [Xclusters_track1, Xclusters_track10, Xclusters_track20]

# Set up function to easily be able to print out name of df being viewed
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

for df_slice in slice_list:
    print('Unique values of hist_user_behavior_n_seekfwd in {}:\n'.format(get_df_name(df_slice)), 
          df_slice['hist_user_behavior_n_seekfwd'].value_counts().compute())

Alright, this information tells us how we'll want to split up this variable into chunks, specific to each df slice.
I actually think that for all of them, it would be wisest to simply make them 0 times, 1-2 times, and 3+ times. We really don't need any more granularity than that.

In [ ]:
# Divide up hist_user_behavior_n_seekfwd in all 3 df slices at once

Xclusters_track1 = Xclusters_track1.compute()
Xclusters_track10 = Xclusters_track10.compute()
Xclusters_track20 = Xclusters_track20.compute()

slice_list = [Xclusters_track1, Xclusters_track10, Xclusters_track20]

for df_slice in slice_list:
    df_slice.loc[df_slice['hist_user_behavior_n_seekfwd'] >= 3, 'hist_user_behavior_n_seekfwd'] = '3+'
    df_slice.loc[(df_slice['hist_user_behavior_n_seekfwd'] == 1) | (
        df_slice['hist_user_behavior_n_seekfwd'] == 2), 'hist_user_behavior_n_seekfwd'] = '1 to 2'
    

In [ ]:
# See if that worked
for df_slice in slice_list:
    print('Unique values of hist_user_behavior_n_seekfwd in {}:\n'.format(get_df_name(df_slice)), 
          df_slice['hist_user_behavior_n_seekfwd'].value_counts())

Great, now when we're ready to get dummies, we'll be able to get them for each of these 3 values.

Now we'll easily do the same with hist_user_behavior_n_seekback

In [ ]:
# Check we still want the same intervals for seekback

for df_slice in slice_list:
    print('Unique values of hist_user_behavior_n_seekback in {}:\n'.format(get_df_name(df_slice)), 
          df_slice['hist_user_behavior_n_seekback'].value_counts())

In [ ]:
# Yep, those intervals still make the most sense to me.

#Recode seekback
for df_slice in slice_list:
    df_slice.loc[df_slice['hist_user_behavior_n_seekback'] >= 3, 'hist_user_behavior_n_seekback'] = '3+'
    df_slice.loc[(df_slice['hist_user_behavior_n_seekback'] == 1) | (
        df_slice['hist_user_behavior_n_seekback'] == 2), 'hist_user_behavior_n_seekback'] = '1 to 2'

# Check it worked
for df_slice in slice_list:
    print('Unique values of hist_user_behavior_n_seekback in {}:\n'.format(get_df_name(df_slice)), 
          df_slice['hist_user_behavior_n_seekback'].value_counts())

Nice, now what we need to do is deal with hour_of_day, and we should be set. Similarly, we don't need the granularity of having every single hour in the day, so we're going to find some meaningful buckets for that and get dummies for those.

In [ ]:
# Print hour of day unique values to think about how we want to divide it up
for df_slice in slice_list:
    print('Unique values of hour_of_day in {}:\n'.format(get_df_name(df_slice)), 
          df_slice['hour_of_day'].value_counts())

I think it makes the most sense to just have buckets of morning, afternoon, night, and, early hours of the morning. Let's see what that would look like.
early = 1-5
morning = 6-11
afternoon = 12-18
night = 19-23, 0

I think this covers it and is a division that will be more meaningful than looking at every individual hour, and this will allow getting dummy variables to go a lot more smoothly as well.

In [ ]:
# For this variable, it will be more efficient to apply a recoding function, rather than use .loc a bunch of times

def recode_value(row, col_name, recode_dict):
    return recode_dict[row[col_name]]

def recode_var(df, col_name, recode_dict):
    df[col_name] = df.apply(lambda row: recode_value(row, col_name, recode_dict), axis=1)
    
for df_slice in slice_list:     
    recode_var(df_slice, 'hour_of_day', {
        1: 'early', 
        2: 'early', 
        3: 'early', 
        4: 'early', 
        5: 'early',
        6: 'morning',
        7: 'morning',
        8: 'morning',
        9: 'morning',
        10: 'morning',
        11: 'morning',
        12: 'afternoon',
        13: 'afternoon',
        14: 'afternoon',
        15: 'afternoon',
        16: 'afternoon',
        17: 'afternoon',
        18: 'afternoon',
        19: 'night',
        20: 'night',
        21: 'night',
        22: 'night',
        23: 'night',
        0: 'night'
    })


In [ ]:
# Check to make sure it worked like I meant it to
for df_slice in slice_list:
    print('Unique values of hour_of_day in {}:\n'.format(get_df_name(df_slice)), 
          df_slice['hour_of_day'].value_counts())

In [ ]:
# I believe all columns are now ready to become dummies, but I'll double-check by looking at the variables
Xclusters_track10.info()

Yes, everything is either binary or in a form that I'm comfortable with for getting dummies.

In [ ]:
# List of columns that need to become dummies (specifying because if I don't, it will only dummify objects)
dummies_list = [
    'hist_user_behavior_n_seekfwd',
    'hist_user_behavior_n_seekback',
    'month',
    'hour_of_day',
    'skip_1', 
    'skip_2',
    'context_switch',
    'no_pause_before_play',
    'hist_user_behavior_is_shuffle',
    'premium',
    'context_type_num',
    'reason_start_num',
    'skipped'
]


# Get the dummies in a separate df; I won't drop_first, because I want to see everything for right now
Xclusters_track1_dummies = pd.get_dummies(Xclusters_track1, columns=dummies_list)

In [ ]:
Xclusters_track1_dummies.info()

In [ ]:
Xclusters_track1.context_switch.value_counts()

In [ ]:
Xclusters_track1.no_pause_before_play.value_counts()

Excellent. Now we'll go ahead and get dummies for the other 2 slices (tracks 10 and 20) and then rename columns so they're more meaningful/easier to interpret. It's important to note that there is only a context_switch_0 for context_switch. Since this df is the first track in the listening session only, *no one* context switched to get to the current track, because that would be impossible, as they weren't listening to music prior to this point. Also notable, no_pause_before_play displays almost the same phenomenon, as most people wouldn't pause when they're just starting their listening session.

In [ ]:
# Make sure track10 df matches
Xclusters_track10.info()

In [ ]:
# Now get its dummies in a separate df
Xclusters_track10_dummies = pd.get_dummies(Xclusters_track10, columns=dummies_list)

In [ ]:
Xclusters_track10_dummies.info()

In [ ]:
Xclusters_track10.reason_start_num.value_counts()

Great, that worked. It's important to note here that the Xclusters_track10 slice did not have any incidences of a code 6 in reason_start_num, even before getting dummies, as evidenced by the above value_counts cell. Just wanted to double-check.

In [ ]:
# Check track20 slice
Xclusters_track20.info()

In [ ]:
# Get its dummies in a separate df
Xclusters_track20_dummies = pd.get_dummies(Xclusters_track20, columns=dummies_list)

In [ ]:
Xclusters_track20_dummies.info()

In [ ]:
Xclusters_track20.reason_start_num.value_counts()

Again, with the 20th track, there were no incidences of code 6 (popup) in the reason_start_num variable. That code only exists in the track 1 dataset, which makes sense, as people might begin their session from a spotify pop-up window, but while they're currently in a listening session, that wouldn't be the reason a track starts.

Now there are a few dummy columns I believe I genuinely don't need, but I want to check what's in them before I drop them.

In [ ]:
print(Xclusters_track20_dummies.skip_1_0.value_counts)
print(Xclusters_track20_dummies.skip_1_1.value_counts)

Right so for these, since they're just binary, we only need one (in the spirit of drop_first, though I didn't want to use the drop_first param, since I got dummies for the enire df). I've checked that they exactly mirror each other, so I can get rid of the **skip_1_0** var.

In [ ]:
print(Xclusters_track20_dummies.skip_2_0)
print(Xclusters_track20_dummies.skip_2_1)

In [ ]:
print(Xclusters_track20.skip_2)

Ditto the skip_1 var. I'll remove the **skip_2_0** var here as well. Skip_2_1 is the original variable. I'm realizing we didn't actually need to get dummies for these, but oh well. Now we know for sure.

In [ ]:
# Let's do the same checking for context_switch
print(Xclusters_track20_dummies.context_switch_0)
print(Xclusters_track20_dummies.context_switch_1)

Great, I feel fine about removing **context_switch_0** also.

In [ ]:
#no_pause_before_play will likely be the same, but let's look at it
print(Xclusters_track20_dummies.no_pause_before_play_0)
print(Xclusters_track20_dummies.no_pause_before_play_1)

In [ ]:
print(Xclusters_track20.no_pause_before_play)

Alright, so for this one, I'm actually going to remove **no_pause_before_play_1**, even though it's the original variable, because the way this one was originally encoded by spotify is sort of reversed, as the 1s in no_pause_1 actually mean there was **no** pause. That's confusing, because it's sort of reversed logic. However, in no_pause_0, the 0s mean the no, the user didn't pause, which **does** make sense. That's why we'll keep no_pause_0. Later we'll rename it so it tells us exactly what it is.

In [ ]:
# I think hist_user_behavior_is_shuffle will be more straightforward, but we'll still check
print(Xclusters_track20_dummies.hist_user_behavior_is_shuffle_0)
print(Xclusters_track20_dummies.hist_user_behavior_is_shuffle_1)

In [ ]:
print(Xclusters_track20.hist_user_behavior_is_shuffle)

Here, we'll drop **hist_user_behavior_is_shuffle_0**, because that's the reversed variable. shuffle_1 is the original var.

In [ ]:
# Check premium
print(Xclusters_track20_dummies.premium_0)
print(Xclusters_track20_dummies.premium_1)

In [ ]:
print(Xclusters_track20.premium)

Remove **premium_0**. 1 is the original variable.

In [ ]:
# Last one: skipped
print(Xclusters_track20_dummies.skipped_0)
print(Xclusters_track20_dummies.skipped_1)

In [ ]:
print(Xclusters_track20.skipped)

Remove **skipped_0**. 1 is the correct/original variable.

In [ ]:
# Remove dummies we don't want
dummy_drop = [
    'skipped_0',
    'premium_0',
    'hist_user_behavior_is_shuffle_0',
    'no_pause_before_play_1',
    'context_switch_0',
    'skip_2_0',
    'skip_1_0'
]

Xclusters_track1_dummies = Xclusters_track1_dummies.drop(columns=dummy_drop)
Xclusters_track10_dummies = Xclusters_track10_dummies.drop(columns=dummy_drop)
Xclusters_track20_dummies = Xclusters_track20_dummies.drop(columns=dummy_drop)

In [ ]:
# Check to see correct cols dropped
Xclusters_track1_dummies.info()

Now it's time to rename the dummy columns so we can see what values they actually stand for. I'm going to have to do these separately becuase the first one has some differences from the other 2, as discussed earlier.

In [ ]:
# Rename track 1 dummies df
rename_dict = {
    "hist_user_behavior_n_seekfwd_0": "no_seekfwd",
    "hist_user_behavior_n_seekfwd_1 to 2": "seekfwd_1to2",
    "hist_user_behavior_n_seekfwd_3+": "seekfwd_3+",
    "hist_user_behavior_n_seekback_0": "no_seekback",
    "hist_user_behavior_n_seekback_1 to 2": "seekback_1to2",
    "hist_user_behavior_n_seekback_3+": "seekback_3+",
    "month_1": "jan",
    "month_2": "feb",
    "month_3": "march",
    "month_4": "april",
    "month_5": "may",
    "month_6": "june",
    "month_7": "july",
    "month_8": "aug",
    "month_9": "sept",
    "month_11": "nov",    # there is no october in the datatsets
    "month_12": "dec",
    "hour_of_day_afternoon": "afternoon",
    "hour_of_day_early": "early_morn",
    "hour_of_day_morning": "morning",
    "hour_of_day_night": "night",
    "skip_1_1": "skip_v_soon", 
    "skip_2_1": "skip_fairly_soon",
    "no_pause_before_play_0": "paused_before_play",
    "hist_user_behavior_is_shuffle_1": "on_shuffle",
    "premium_1": "premium",
    "context_type_num_0": "context_catalog",
    "context_type_num_1": "context_charts",
    "context_type_num_2": "context_editorial_playlist",
    "context_type_num_3": "context_personal_playlist",
    "context_type_num_4": "context_radio",
    "context_type_num_5": "context_user_collec",
    "reason_start_num_0": "appload",
    "reason_start_num_1": "start_backbutton",
    "reason_start_num_2": "start_clickrow",
    "reason_start_num_3": "start_endplay",
    "reason_start_num_4": "start_fwdbutton",
    "reason_start_num_5": "start_playbutton",
    "reason_start_num_6": "popup",
    "reason_start_num_7": "start_remote",
    "reason_start_num_8": "start_trackdone",
    "reason_start_num_9": "start_uriopen",
    "skipped_1": "skipped"
}

Xclusters_track1_dummies = Xclusters_track1_dummies.rename(columns=rename_dict)

In [ ]:
Xclusters_track1_dummies.info()

In [ ]:
# Rename columns in track10 and track20 dummies df

rename_dict = {
    "hist_user_behavior_n_seekfwd_0": "no_seekfwd",
    "hist_user_behavior_n_seekfwd_1 to 2": "seekfwd_1to2",
    "hist_user_behavior_n_seekfwd_3+": "seekfwd_3+",
    "hist_user_behavior_n_seekback_0": "no_seekback",
    "hist_user_behavior_n_seekback_1 to 2": "seekback_1to2",
    "hist_user_behavior_n_seekback_3+": "seekback_3+",
    "month_1": "jan",
    "month_2": "feb",
    "month_3": "march",
    "month_4": "april",
    "month_5": "may",
    "month_6": "june",
    "month_7": "july",
    "month_8": "aug",
    "month_9": "sept",
    "month_11": "nov",    # there is no october in the datatsets
    "month_12": "dec",
    "hour_of_day_afternoon": "afternoon",
    "hour_of_day_early": "early_morn",
    "hour_of_day_morning": "morning",
    "hour_of_day_night": "night",
    "skip_1_1": "skip_v_soon", 
    "skip_2_1": "skip_fairly_soon",
    "context_switch_1": "switched_context",
    "no_pause_before_play_0": "paused_before_play",
    "hist_user_behavior_is_shuffle_1": "on_shuffle",
    "premium_1": "premium",
    "context_type_num_0": "context_catalog",
    "context_type_num_1": "context_charts",
    "context_type_num_2": "context_editorial_playlist",
    "context_type_num_3": "context_personal_playlist",
    "context_type_num_4": "context_radio",
    "context_type_num_5": "context_user_collec",
    "reason_start_num_0": "appload",
    "reason_start_num_1": "start_backbutton",
    "reason_start_num_2": "start_clickrow",
    "reason_start_num_3": "start_endplay",
    "reason_start_num_4": "start_fwdbutton",
    "reason_start_num_5": "start_playbutton",
    "reason_start_num_7": "start_remote",
    "reason_start_num_8": "start_trackdone",
    "reason_start_num_9": "start_uriopen",
    "skipped_1": "skipped"
}

Xclusters_track10_dummies = Xclusters_track10_dummies.rename(columns=rename_dict)
Xclusters_track20_dummies = Xclusters_track20_dummies.rename(columns=rename_dict)

In [ ]:
# Check renamed correctly
print(Xclusters_track10_dummies.info())
print(Xclusters_track20_dummies.info())

Now let's make some clusters! I want to start with HDBSCAN and see where that gets us. This is nice, because it will determine the optimal number for eps for us. 

We will cluster the 3 slices separately and see what the clusters can tell us about each one. 

In [ ]:
# First, we'll just do track1

# Turn the df into an array
track1_array = np.array(Xclusters_track1_dummies.values)

In [ ]:
print(track1_array)

In [ ]:
# Standardize track1
scaler = StandardScaler()
track1_array_std = scaler.fit_transform(track1_array)

In [ ]:
# Check it out
track1_array_std

In [ ]:
# Make it a dask array to clusters run faster
track1_darray = da.from_array(track1_array_std, chunks=(10000,10000))

In [ ]:
hd_clusters = hdbscan.HDBSCAN(min_samples=23)

In [ ]:
# Get cluster predictions
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels
hd_labels = hd_clusters.labels_
print(hd_labels)

In [ ]:
# How many clusters are there?
hd_labels.max()

Okay, so 1032 clusters is obviously way too many. We need to increase the min_samples and try again. We'll go to 100 and see if we can find a good window.

In [ ]:
# hdbscan with 50 min_samples
hd_clusters = hdbscan.HDBSCAN(min_samples=50)

In [ ]:
# Get cluster predictions with 50 min_samples
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for 50 min_samples
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

Okay, still far too many clusters. We'll try with 100 min_samples. That may be closer to what we'd need.

In [ ]:
# hdbscan with 100 min_samples
hd_clusters = hdbscan.HDBSCAN(min_samples=100)

In [ ]:
# Get cluster preds with 100 min_samples
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for 100 min_samples
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# hdbsan with 75 min_samples
hd_clusters = hdbscan.HDBSCAN(min_samples=75)

In [ ]:
# Get cluster preds with 75 min_samples
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for 75 min_samples
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# hdbscan with manhattan distance
hd_clusters = hdbscan.HDBSCAN(metric='manhattan')

In [ ]:
# Get cluster preds with manhattan distance
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Okay let's make this manhattan distance version better with more params
hd_clusters = hdbscan.HDBSCAN(min_samples=43, min_cluster_size=10, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 43 + min_clust_size 10
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 43 + min_clust_size 10
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Manhattan again, reducing min_clust_size & increasing min_samples slightly
hd_clusters = hdbscan.HDBSCAN(min_samples=50, min_cluster_size=8, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 50 + min_clust_size 8
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 50 + min_clust_size 8
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Manhattan again, reducing min_clust_size 10 & min_samples 50
hd_clusters = hdbscan.HDBSCAN(min_samples=50, min_cluster_size=10, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 50 + min_clust_size 10
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 50 + min_clust_size 10
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 15 & min_samples at 50
hd_clusters = hdbscan.HDBSCAN(min_samples=50, min_cluster_size=15, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 50 + min_clust_size 15
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 50 + min_clust_size 15
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 25 & min_samples at 50
hd_clusters = hdbscan.HDBSCAN(min_samples=50, min_cluster_size=25, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 50 + min_clust_size 25
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 50 + min_clust_size 15
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 30 & min_samples to 65
hd_clusters = hdbscan.HDBSCAN(min_samples=65, min_cluster_size=30, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 65 + min_clust_size 30
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 65 + min_clust_size 30
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 35 & min_samples to 75
hd_clusters = hdbscan.HDBSCAN(min_samples=75, min_cluster_size=35, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 75 + min_clust_size 35
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 75 + min_clust_size 35
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 50 & min_samples to 85
hd_clusters = hdbscan.HDBSCAN(min_samples=85, min_cluster_size=50, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 85 + min_clust_size 50
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 85 + min_clust_size 50
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 60 & leaving min_samples at 85
hd_clusters = hdbscan.HDBSCAN(min_samples=85, min_cluster_size=60, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 85 + min_clust_size 60
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 85 + min_clust_size 60
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 65 & reducing min_samples to 75
hd_clusters = hdbscan.HDBSCAN(min_samples=75, min_cluster_size=65, metric='manhattan')

In [ ]:
#  Increasing min_clust_size to 65 & reducing min_samples to 75
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 75 + min_clust_size 65
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 50 & min_samples to 90
hd_clusters = hdbscan.HDBSCAN(min_samples=90, min_cluster_size=50, metric='manhattan')

In [ ]:
# Get cluster preds with manhattan + min_samples 90 + min_clust_size 50
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 90 + min_clust_size 50
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 100 & leave min_samples at 90
hd_clusters = hdbscan.HDBSCAN(min_samples=90, min_cluster_size=100, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 90 & clust_size 100
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 90 + min_clust_size 100
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 150 & increasing min_samples to 100
hd_clusters = hdbscan.HDBSCAN(min_samples=100, min_cluster_size=150, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 100 & clust_size 150
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 100 + min_clust_size 150
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 175 & keeping min_samples at 100
hd_clusters = hdbscan.HDBSCAN(min_samples=100, min_cluster_size=175, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 100 & clust_size 175
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 100 + min_clust_size 150
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 200 & keeping min_samples at 100
hd_clusters = hdbscan.HDBSCAN(min_samples=100, min_cluster_size=200, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 100 & clust_size 200
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 100 + min_clust_size 200
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Keeping min_clust_size at 200 & increasing min_samples to 125
hd_clusters = hdbscan.HDBSCAN(min_samples=125, min_cluster_size=200, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 125 & clust_size 200
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 100 + min_clust_size 200
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 225 & increasing min_samples to 125
hd_clusters = hdbscan.HDBSCAN(min_samples=125, min_cluster_size=225, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 125 & clust_size 225
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 125 + min_clust_size 225
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 275 & keeping min_samples at 125
hd_clusters = hdbscan.HDBSCAN(min_samples=125, min_cluster_size=275, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 125 & clust_size 275
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 125 + min_clust_size 275
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 350 & keeping min_samples at 125
hd_clusters = hdbscan.HDBSCAN(min_samples=125, min_cluster_size=350, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 125 & clust_size 350
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 125 + min_clust_size 350
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 450 & increasing min_samples at 175
hd_clusters = hdbscan.HDBSCAN(min_samples=175, min_cluster_size=450, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 175 & clust_size 450
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 175 + min_clust_size 450
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 500 & increasing min_samples at 200
hd_clusters = hdbscan.HDBSCAN(min_samples=200, min_cluster_size=500, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 200 & clust_size 500
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 200 + min_clust_size 500
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Keeping min_clust_size at 500 & increasing min_samples to 300
hd_clusters = hdbscan.HDBSCAN(min_samples=300, min_cluster_size=500, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 300 & clust_size 500
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 300 + min_clust_size 500
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 550 & increasing min_samples at 350
hd_clusters = hdbscan.HDBSCAN(min_samples=350, min_cluster_size=550, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 350 & clust_size 550
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 350 + min_clust_size 550
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 600 & increasing min_samples at 375
hd_clusters = hdbscan.HDBSCAN(min_samples=375, min_cluster_size=600, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 375 & clust_size 600
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 375 + min_clust_size 600
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 625 & keeping min_samples at 375
hd_clusters = hdbscan.HDBSCAN(min_samples=375, min_cluster_size=625, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 375 & clust_size 625
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 375 + min_clust_size 625
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Keeping min_clust_size at 625 & increasing min_samples to 425
hd_clusters = hdbscan.HDBSCAN(min_samples=425, min_cluster_size=625, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 425 & clust_size 625
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 425 + min_clust_size 625
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 650 & keeping min_samples at 425
hd_clusters = hdbscan.HDBSCAN(min_samples=425, min_cluster_size=650, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 425 & clust_size 650
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 425 + min_clust_size 650
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 700 & increasing min_samples to 475
hd_clusters = hdbscan.HDBSCAN(min_samples=475, min_cluster_size=700, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 475 & clust_size 700
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 475 + min_clust_size 700
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 900 & increasing min_samples to 500
hd_clusters = hdbscan.HDBSCAN(min_samples=500, min_cluster_size=900, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 500 & clust_size 900
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 500 + min_clust_size 900
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 950 & increasing min_samples to 550
hd_clusters = hdbscan.HDBSCAN(min_samples=550, min_cluster_size=950, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 550 & clust_size 950
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Check cluster labels for manhattan + min_samples 550 + min_clust_size 950
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1000 & increasing min_samples to 650
hd_clusters = hdbscan.HDBSCAN(min_samples=650, min_cluster_size=1000, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 650 & clust_size 1000
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 650 + min_clust_size 1000
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1050 & increasing min_samples to 800
hd_clusters = hdbscan.HDBSCAN(min_samples=800, min_cluster_size=1050, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 800 & clust_size 1050
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 800 + min_clust_size 1050
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1150 & increasing min_samples to 950
hd_clusters = hdbscan.HDBSCAN(min_samples=950, min_cluster_size=1150, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 950 & clust_size 1150
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 950 + min_clust_size 1150
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1200 & keeping min_samples at 950
hd_clusters = hdbscan.HDBSCAN(min_samples=950, min_cluster_size=1200, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 950 & clust_size 1200
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 950 + min_clust_size 1200
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1400 & keeping min_samples at 1000
hd_clusters = hdbscan.HDBSCAN(min_samples=1000, min_cluster_size=1400, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 1000 & clust_size 1400
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 1000 + min_clust_size 1400
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1500 & increasing min_samples to 1100
hd_clusters = hdbscan.HDBSCAN(min_samples=1100, min_cluster_size=1500, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 1100 & clust_size 1500
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 1100 + min_clust_size 1500
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Increasing min_clust_size to 1600 & increasing min_samples to 1200
hd_clusters = hdbscan.HDBSCAN(min_samples=1200, min_cluster_size=1600, metric='manhattan')

In [ ]:
# Get preds with manhattan + min_samples 1200 & clust_size 1600
hd_preds = hd_clusters.fit_predict(track1_darray)

In [ ]:
# Cluster labels for manhattan + min_samples 1200 + min_clust_size 1600
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Figure out how much noise there is in this 11-cluster solution
no_noise_labels = []
for label in hd_labels:
    if label != -1:
        no_noise_labels.append(label)
print(len(no_noise_labels))

print('{}% of the data is not considered noise: '.format((len(no_noise_labels) / len(track1_darray))*100))

Okay, half and half is not too bad!

In [ ]:
# Set up vectorizer with colors for plotting
colors = [
    'royalblue', 
    'maroon', 
    'forestgreen', 
    'mediumorchid', 
    'tan', 
    'deeppink', 
    'olive', 
    'goldenrod', 
    'lightcyan', 
    'navy',
    'chartreuse'
]

vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

In [ ]:
# Plot clusters
plt.scatter(track1_darray[:,0], track1_darray[:,1], c=vectorizer(hd_labels))

In [ ]:
def plot_clusters(data, labels, args, kwds):
    labels = labels   
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0,0.0,0.0) for x in labels]
    plot_kwds={'alpha':0.25, 's':60, 'linewidths':0}
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by HDBSCAN', fontsize=24)

    
plot_clusters(track1_darray, hd_labels, (), {'n_clusters':11}) 

In [ ]:
# Hm, it looks like there may be too many points to view properly? 
#I'll take a random sample & see if it plots better

# data_tup_list = []
# label_tup_list = []

# for index1, val in enumerate(track1_darray.compute()):
#     data_tup = (val, index1)
#     data_tup_list.append(data_tup)

# for index2, label in enumerate(hd_labels):
#     label_tup = (label, index2)
#     label_tup_list.append(label_tup)


In [ ]:
# print(data_tup_list[0:10])
# print(label_tup_list[0:10])

In [ ]:
# Need to maintain context of random sample of track1 data, so this sample can be plotted with correct cluster labels

# 1. create list of the dataset's indices, then randomly sample from that
# 2. loop through the random sample to get each index and save it in a variable
# 3. save the corresponding cluster label indices in another variable 
# 4. append each randomly sampled datapoint (in order of indices) to a list 
# 5. append each corresponding cluster label to its own list (also in same order)
# 6. return the data and its correct label

data_index_list = np.arange(0, len(track1_darray), 1)
rsample = np.random.choice(data_index_list, 500, replace=False)
 
plotting_data = []
plotting_data_labels = []

for index in rsample:
    data = track1_darray[index].compute()
    label = hd_labels[index]
    plotting_data.append(data)
    plotting_data_labels.append(label)
    print(label, data)


In [ ]:
print(plotting_data)
print(plotting_data_labels)

In [ ]:
# plotting_data is currently multiple arrays; we need to make it one
plotting_data_array = np.array(plotting_data)
plotting_data_array

In [ ]:
# Now plot the random sample using the vectorizer 
plt.figure(figsize=(15, 10))
plt.scatter(plotting_data_array[:,0], plotting_data_array[:,1], c=vectorizer(plotting_data_labels))

Okay, now that we've got our solution for the track 1 dataset, we'll repeat this clustering solution on the track 10 and track 20 datasets. However, it will be a lot faster this time, since we already know (more or less) the best parameters to use.

In [ ]:
# we'll just do track10 now

# Turn the df into an array
track10_array = np.array(Xclusters_track10_dummies.values)

In [ ]:
# Standardize track10
scaler = StandardScaler()
track10_array_std = scaler.fit_transform(track10_array)

In [ ]:
# Check out the new array
track10_array_std

In [ ]:
# Make it a dask array to clusters run faster
track10_darray = da.from_array(track10_array_std, chunks=(10000,10000))

In [ ]:
# Alright, let's try with the params that gave us our 11 cluster solution on track1:
hd_clusters = hdbscan.HDBSCAN(min_samples=1200, min_cluster_size=1600, metric='manhattan')

In [ ]:
hd_preds = hd_clusters.fit_predict(track10_darray)

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1200 + min_clust_size 1600
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

Okay, this gives us 14. Suppose we make it a bit smaller.

In [ ]:
hd_clusters = hdbscan.HDBSCAN(min_samples=1250, min_cluster_size=1650, metric='manhattan')

In [ ]:
hd_preds = hd_clusters.fit_predict(track10_darray)

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1250 + min_clust_size 1650
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Continue trying to make fewer clusters
hd_clusters = hdbscan.HDBSCAN(min_samples=1350, min_cluster_size=1650, metric='manhattan')

In [ ]:
hd_preds = hd_clusters.fit_predict(track10_darray)

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1350 + min_clust_size 1650
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
hd_clusters = hdbscan.HDBSCAN(min_samples=1375, min_cluster_size=1675)

In [ ]:
hd_preds = hd_clusters.fit_predict(track10_darray)

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1375 + min_clust_size 1675
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
hd_clusters = hdbscan.HDBSCAN(min_samples=1300, min_cluster_size=1650)

In [ ]:
hd_preds = hd_clusters.fit_predict(track10_darray)

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1300 + min_clust_size 1650
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
hd_clusters = hdbscan.HDBSCAN(min_samples=1400, min_cluster_size=1650)

In [ ]:
hd_preds = hd_clusters.fit_predict(track10_darray)

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1400 + min_clust_size 1650
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()

In [ ]:
# Cluster labels for track10 manhattan + min_samples 1400 + min_clust_size 1650
hd_labels = hd_clusters.labels_
print(hd_labels)

# How many clusters are there?
hd_labels.max()
hd_labels.max()

Alright, well I guess I am just going to keep this one with the 12 cluster solution, since it reallly did not want to fit into an 11 cluster solution, no matter how I modified the parameters.

In [ ]:
# So right now let's make variables for each of the chosen solutions 
# Here are the final variables for the track1 HDBSCAN solution:

track1_hdbscan = hdbscan.HDBSCAN(min_samples=1450, min_cluster_size=1700)
track1_preds = track1_hdbscan.fit_predict(track1_darray)
track1_hd_labels = track1_hdbscan.labels_

# How many clusters are there?
track1_hd_labels.max()

In [ ]:
# Here are the final variables for the track10 HDBSCAN solution:

track10_hdbscan = hdbscan.HDBSCAN(min_samples=1450, min_cluster_size=1750)
track10_preds = track10_hdbscan.fit_predict(track10_darray)
track10_hd_labels = track10_hdbscan.labels_

# How many clusters are there?
track10_hd_labels.max()

In [ ]:
# Okay how about we visualize the track10 clusters

plot_clusters(track10_darray, track10_hd_labels, (), {'n_clusters':12}) 

In [ ]:
# Make track20 dataset into array
track20_array = np.array(Xclusters_track20_dummies.values)

In [ ]:
# Standardize the track20 array
scaler = StandardScaler()
track20_array_std = scaler.fit_transform(track20_array)

In [ ]:
track20_array_std

In [ ]:
# Make it a dask array to clusters run faster
track20_darray = da.from_array(track20_array_std, chunks=(10000,10000))

In [ ]:
# Now let's try with track20

track20_hdbscan = hdbscan.HDBSCAN(min_samples=1600, min_cluster_size=1700)
track20_preds = track20_hdbscan.fit_predict(track20_darray)
track20_hd_labels = track20_hdbscan.labels_

#How many clusters are there?
track20_hd_labels.max()

In [ ]:
# Okay how about we visualize the track20 clusters

plot_clusters(track20_darray, track20_hd_labels, (), {'n_clusters':11}) 

Okay well the cluster plots all look the same, which was to be expected, since they all contain very similar data.

The next thing to do is run a multivariate linear regression so we can look at the coefficients for each of the clusters and determine which are most influential. Then we'll also do some barplots and such and see how the clusters play out across different variables.

In [ ]:
# We'll use train test split to divide up dataset just for these cluster regressions

# First is the track1 clusters 
X = track1_darray
y = track1_hd_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=60)

In [ ]:
cluster_clf1 = LogisticRegression(multi_class ='multinomial', solver ='newton-cg')
cluster_clf1_fit = cluster_clf.fit(X_train, y_train)

In [ ]:
y_pred = cluster_clf1_fit.predict(X_test)

In [ ]:
# Now a confusion matrix for the track1 clusters' logistic regression
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

In [ ]:
print(cluster_clf1_fit.coef_)
print(cluster_clf1_fit.classes_)

In [822]:
#Okay, actually I think I want to do this regression with the df, not the array

# First is the track1 clusters - we need to standardize the df first

scaler = StandardScaler()
X1 = scaler.fit_transform(Xclusters_track1_dummies)
y1 = track1_hd_labels

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=.2, random_state=60)

In [823]:
cluster_clf1 = LogisticRegression(multi_class ='multinomial', solver ='newton-cg')
cluster_clf1_fit = cluster_clf1.fit(X1_train, y1_train)

In [824]:
y1_pred = cluster_clf1_fit.predict(X1_test)

In [825]:
# Now a confusion matrix for the track1 clusters' logistic regression
cm1 = confusion_matrix(y1_test, y1_pred)
print(cm1)

[[7281   24    6    6   60  144  180    0    0    0    0    1    0]
 [  30  688    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0  589    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0 1024    0    0    0    0    0    0    0    0    0]
 [  20    0    0    0 1210    0    0    0    0    0    0    0    0]
 [   8    0    0    0    0  999    0    0    0    0    0    0    0]
 [  64    0    0    0    0    0 1187    0    0    0    0    0    0]
 [   4    0    0    0    0    0    0  860    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0  732    0    0    0    0]
 [   2    0    0    0    0    0    0    0    0  487    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0 1251    0    0]
 [   6    0    0    0    0    0    0    0    0    0    0  634    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0  438]]


In [752]:
# Check loss function for track1
# log_loss(X_test, y_pred)

In [826]:
# Look at coefficients for each cluster class. The cluster classes themselves are also shown at the bottom
print(cluster_clf1_fit.coef_)
print(cluster_clf1_fit.classes_)

[[-1.88336859e+00  1.82063773e+00  5.21517175e-01 -1.70521913e+00
   1.40973889e+00  1.10128542e+00 -5.34213526e-08 -8.16604353e-08
   1.85997396e-07  1.75659156e-07  4.88381480e-02  3.11384177e-01
  -2.96000063e-01  6.39938147e-03 -8.16604353e-08 -8.16604358e-08
  -2.67484995e+00  4.27468365e+00  1.87423574e+00 -1.18817088e+00
   1.07651579e+00  5.69494967e-01 -1.27214354e-01  1.39169103e+00
  -4.28726520e+00  2.69735508e-03  1.60243980e+00  1.29637345e+00
   1.69231829e+00  1.86322777e+00 -3.11109579e+00  5.11663334e-01
   3.16567029e-01 -3.67531828e-01  2.39752745e-01  3.50437825e-01
   9.48400689e-01  3.36975618e-02  4.22184202e-01 -8.30653496e-01
   2.21432707e-01 -4.29441843e-01]
 [-5.98957951e-01  6.19007070e-01 -1.01372333e-01 -3.80409963e+00
   3.99266348e+00 -3.04092975e-03  6.26271097e-09  7.54225368e-09
   6.79446353e-09  8.83371788e-09  1.30565666e-08  5.05342332e-08
   9.93738653e-04 -6.39917267e-03  7.54225368e-09  7.54225371e-09
  -8.91087033e-01 -2.45506898e-02  1.3133

In [827]:
#Okay, let's look at these coeffs in an easier to interpret format

for cluster in range(len(cluster_clf1_fit.coef_)):
    print('For cluster {}: '.format(cluster-1))
    for coef in range(len(cluster_clf1_fit.coef_[cluster])):
        print('   ', Xclusters_track1_dummies.columns[coef], '=', cluster_clf1_fit.coef_[cluster][coef])
        

For cluster -1: 
    no_seekfwd = -1.883368594611479
    seekfwd_1to2 = 1.820637731109978
    seekfwd_3+ = 0.5215171751326356
    no_seekback = -1.7052191261259755
    seekback_1to2 = 1.4097388925315477
    seekback_3+ = 1.1012854249493866
    jan = -5.342135256998413e-08
    feb = -8.166043531598256e-08
    march = 1.8599739618378646e-07
    april = 1.756591556720329e-07
    may = 0.04883814804667897
    june = 0.3113841766024214
    july = -0.2960000627185701
    aug = 0.006399381473804537
    sept = -8.166043531598256e-08
    nov = -8.166043577111878e-08
    afternoon = -2.6748499458001564
    early_morn = 4.274683651433285
    morning = 1.8742357432931056
    night = -1.1881708804585434
    skip_v_soon = 1.0765157907600862
    skip_fairly_soon = 0.5694949672127005
    paused_before_play = -0.12721435446938606
    on_shuffle = 1.3916910334984784
    premium = -4.287265195858187
    context_catalog = 0.002697355082030879
    context_charts = 1.6024397965617736
    context_editorial_p

    start_backbutton = -0.8610794088239095
    start_clickrow = 0.5354530808351617
    start_endplay = -6.2047226451632766e-06
    start_fwdbutton = 1.2860694287416286
    start_playbutton = -0.13155830277131408
    popup = -9.459201474474046e-07
    start_remote = -0.0002710251611446685
    start_trackdone = -0.5781850705405729
    start_uriopen = -0.033653256992699034
    skipped = -1.0891830448714037
For cluster 5: 
    no_seekfwd = 0.06024220081779903
    seekfwd_1to2 = -0.060679698437915
    seekfwd_3+ = -0.0003532576340789105
    no_seekback = 0.022416983794567604
    seekback_1to2 = -0.02350295596780136
    seekback_3+ = -5.521981805181894e-05
    jan = 4.058020755422366e-09
    feb = 4.0801001349650515e-09
    march = 2.9054282004203263e-09
    april = 3.2773942270354124e-09
    may = -3.76848706940546e-09
    june = -1.138235910692874e-06
    july = 1.0273421525942734e-06
    aug = 9.027309868990266e-09
    sept = 4.0801001349650515e-09
    nov = 4.0801001686833155e-09
    aft

In [828]:
#  I want to see the maximum coefs for each cluster along with corresponding column names:
for cluster in range(len(cluster_clf1_fit.coef_)):
    max_seen = float("-inf")
    column_name = None
    for index, coef in enumerate(cluster_clf1_fit.coef_[cluster]):
        if coef <= max_seen: continue 
        column_name = Xclusters_track1_dummies.columns[index]
        max_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), max_seen, column_name)

Cluster -1 Max Coef:  4.274683651433285 early_morn
Cluster 0 Max Coef:  3.9926634830353347 seekback_1to2
Cluster 1 Max Coef:  5.908169174439731 start_backbutton
Cluster 2 Max Coef:  8.315857555040559 appload
Cluster 3 Max Coef:  6.784157764711629 context_radio
Cluster 4 Max Coef:  6.120455562962183 morning
Cluster 5 Max Coef:  5.05845384413869 context_editorial_playlist
Cluster 6 Max Coef:  6.299106826995431 start_clickrow
Cluster 7 Max Coef:  4.478108411952089 start_clickrow
Cluster 8 Max Coef:  3.8464768831346268 start_trackdone
Cluster 9 Max Coef:  5.404487992119586 start_trackdone
Cluster 10 Max Coef:  6.010811796061196 skip_fairly_soon
Cluster 11 Max Coef:  5.115892059084914 start_fwdbutton


In [829]:
# Print out absolute value maximum coefficients (whether pos or neg)
for cluster in range(len(cluster_clf1_fit.coef_)):
    max_seen = 0
    column_name = None
    for index, coef in enumerate(cluster_clf1_fit.coef_[cluster]):
        abs_coef = abs(coef)
        if abs_coef <= abs(max_seen): continue 
        column_name = Xclusters_track1_dummies.columns[index]
        max_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), max_seen, column_name)

Cluster -1 Max Coef:  -4.287265195858187 premium
Cluster 0 Max Coef:  3.9926634830353347 seekback_1to2
Cluster 1 Max Coef:  5.908169174439731 start_backbutton
Cluster 2 Max Coef:  8.315857555040559 appload
Cluster 3 Max Coef:  6.784157764711629 context_radio
Cluster 4 Max Coef:  6.120455562962183 morning
Cluster 5 Max Coef:  5.05845384413869 context_editorial_playlist
Cluster 6 Max Coef:  6.299106826995431 start_clickrow
Cluster 7 Max Coef:  4.478108411952089 start_clickrow
Cluster 8 Max Coef:  3.8464768831346268 start_trackdone
Cluster 9 Max Coef:  5.404487992119586 start_trackdone
Cluster 10 Max Coef:  6.010811796061196 skip_fairly_soon
Cluster 11 Max Coef:  5.115892059084914 start_fwdbutton


In [830]:
# Print out minimum coefficients (which in this case actually means they are the strongest negative ones)
for cluster in range(len(cluster_clf1_fit.coef_)):
    min_seen = float("inf")
    column_name = None
    for index, coef in enumerate(cluster_clf1_fit.coef_[cluster]):
        if coef >= min_seen: continue 
        column_name = Xclusters_track1_dummies.columns[index]
        min_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), min_seen, column_name)

Cluster -1 Max Coef:  -4.287265195858187 premium
Cluster 0 Max Coef:  -3.8040996339328217 no_seekback
Cluster 1 Max Coef:  -1.7841707874805532 seekfwd_1to2
Cluster 2 Max Coef:  -5.3772292775124875 no_seekfwd
Cluster 3 Max Coef:  -3.491006767169699 context_user_collec
Cluster 4 Max Coef:  -3.1562357774324656 afternoon
Cluster 5 Max Coef:  -3.323561061517099 context_user_collec
Cluster 6 Max Coef:  -4.4889563157622705 start_fwdbutton
Cluster 7 Max Coef:  -4.025792833796309 skip_v_soon
Cluster 8 Max Coef:  -3.467241118974595 skip_fairly_soon
Cluster 9 Max Coef:  -5.051265221248484 skip_v_soon
Cluster 10 Max Coef:  -2.4663383580452645 start_clickrow
Cluster 11 Max Coef:  -2.4862712345285005 start_clickrow


In [831]:
# Now we'll get the odds ratios for track1:

t1_odds = np.exp(cluster_clf1_fit.coef_)
t1_odds

array([[1.52076956e-01, 6.17579569e+00, 1.68458152e+00, 1.81732559e-01,
        4.09488606e+00, 3.00803014e+00, 9.99999947e-01, 9.99999918e-01,
        1.00000019e+00, 1.00000018e+00, 1.05005038e+00, 1.36531364e+00,
        7.43787381e-01, 1.00641990e+00, 9.99999918e-01, 9.99999918e-01,
        6.89171689e-02, 7.18574037e+01, 6.51583744e+00, 3.04778230e-01,
        2.93443752e+00, 1.76737424e+00, 8.80544904e-01, 4.02164504e+00,
        1.37424569e-02, 1.00270100e+00, 4.96513157e+00, 3.65601389e+00,
        5.43205921e+00, 6.44450462e+00, 4.45521090e-02, 1.66806344e+00,
        1.37240823e+00, 6.92441288e-01, 1.27093487e+00, 1.41968899e+00,
        2.58157761e+00, 1.03427176e+00, 1.52528946e+00, 4.35764423e-01,
        1.24786327e+00, 6.50872282e-01],
       [5.49383823e-01, 1.85708317e+00, 9.03596532e-01, 2.22792476e-02,
        5.41990556e+01, 9.96963689e-01, 1.00000001e+00, 1.00000001e+00,
        1.00000001e+00, 1.00000001e+00, 1.00000001e+00, 1.00000005e+00,
        1.00099423e+00,

In [832]:
# I want to print out the maximum odds ratios for each column:
for cluster in range(len(t1_odds)):
    max_seen = float("-inf")
    column_name = None
    for index, oratio in enumerate(t1_odds[cluster]):
        if oratio <= max_seen: continue 
        column_name = Xclusters_track1_dummies.columns[index]
        max_seen = oratio
    print('Cluster {} Max Odds Ratio: '.format(cluster-1), max_seen, column_name)

Cluster -1 Max Odds Ratio:  71.85740372698724 early_morn
Cluster 0 Max Odds Ratio:  54.19905555099232 seekback_1to2
Cluster 1 Max Odds Ratio:  368.03173631604557 start_backbutton
Cluster 2 Max Odds Ratio:  4088.189782271003 appload
Cluster 3 Max Odds Ratio:  883.7354602234809 context_radio
Cluster 4 Max Odds Ratio:  455.0719612150786 morning
Cluster 5 Max Odds Ratio:  157.3470450936046 context_editorial_playlist
Cluster 6 Max Odds Ratio:  544.0857303504048 start_clickrow
Cluster 7 Max Odds Ratio:  88.06792678039766 start_clickrow
Cluster 8 Max Odds Ratio:  46.827792482983284 start_trackdone
Cluster 9 Max Odds Ratio:  222.40231958697484 start_trackdone
Cluster 10 Max Odds Ratio:  407.81424793301414 skip_fairly_soon
Cluster 11 Max Odds Ratio:  166.6493757854578 start_fwdbutton


In [833]:
# Now we'll run a logistic regression on the clusters for track10

scaler = StandardScaler()
X10 = scaler.fit_transform(Xclusters_track10_dummies)
y10 = track10_hd_labels

X10_train, X10_test, y10_train, y10_test = train_test_split(X10, y10, test_size=.2, random_state=60)

In [820]:
# track10_hd_labels

array([-1, 10,  5, ..., -1,  6,  7])

In [821]:
# Xclusters_track10_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89672 entries, 0 to 58119
Data columns (total 42 columns):
no_seekfwd                    89672 non-null uint8
seekfwd_1to2                  89672 non-null uint8
seekfwd_3+                    89672 non-null uint8
no_seekback                   89672 non-null uint8
seekback_1to2                 89672 non-null uint8
seekback_3+                   89672 non-null uint8
jan                           89672 non-null uint8
feb                           89672 non-null uint8
march                         89672 non-null uint8
april                         89672 non-null uint8
may                           89672 non-null uint8
june                          89672 non-null uint8
july                          89672 non-null uint8
aug                           89672 non-null uint8
sept                          89672 non-null uint8
nov                           89672 non-null uint8
afternoon                     89672 non-null uint8
early_morn              

In [834]:
cluster_clf2 = LogisticRegression(multi_class ='multinomial', solver ='newton-cg')
cluster_clf2_fit = cluster_clf2.fit(X10_train, y10_train)

In [835]:
y10_pred = cluster_clf2_fit.predict(X10_test)

In [836]:
# Now a confusion matrix for the track1 clusters' logistic regression
cm2 = confusion_matrix(y10_test, y10_pred)
print(cm2)

[[8725    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 365    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 789    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 659    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 552    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 415    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 931    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [1287    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 381    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 699    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 881    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [1041    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 762    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 448    0    0    0    0    0    0    0    0    0

In [814]:
print(cluster_clf2_fit.coef_)
print(cluster_clf2_fit.classes_)

[[-4.41540289e-03  3.82252311e-03  2.49802228e-03 -5.10462360e-03
   1.41804133e-03  1.41090040e-02 -8.72984593e-03 -3.03266394e-06
   2.69911027e-02  3.54694484e-02  4.15233480e-02  2.82424896e-05
  -2.51020799e-02  2.82060705e-02  2.69585666e-02 -8.62181309e-03
  -5.82052079e-03 -1.52580729e-03  9.06435247e-03 -1.19115762e-04
  -1.93129441e-02  1.50321722e-02  8.34951790e-03 -8.27871357e-03
  -7.13305451e-03 -8.37016141e-03 -7.75456034e-03  6.20821356e-03
   6.31636184e-03 -5.04120087e-03 -7.88081985e-03  6.53674135e-03
   2.22660535e-02 -4.61981797e-03 -8.08662499e-03 -1.46124671e-02
   5.22180141e-03  2.06728460e-02  3.28385393e-02 -3.19003989e-03
   6.88889178e-03 -4.63130943e-03]
 [ 9.99998839e-03 -1.75570634e-02  2.17230031e-02  3.47413383e-02
  -4.23092654e-02  2.31014114e-02 -1.08261904e-04  1.04336686e-06
  -1.36669767e-04 -1.54157524e-04 -2.60170905e-03 -7.99600274e-02
   7.32777057e-02 -7.29151198e-04 -1.28254381e-04 -1.71455373e-04
  -5.91538948e-04  4.71900485e-03  2.3267

In [808]:
#Okay, let's look at these coeffs in an easier to interpret format

for cluster in range(len(cluster_clf2_fit.coef_)):
    print('For cluster {}: '.format(cluster-1))
    for coef in range(len(cluster_clf2_fit.coef_[cluster])):
        print('   ', Xclusters_track10_dummies.columns[coef], '=', cluster_clf2_fit.coef_[cluster][coef])
        

For cluster -1: 
    no_seekfwd = -0.004415402886632337
    seekfwd_1to2 = 0.003822523109801116
    seekfwd_3+ = 0.0024980222815223414
    no_seekback = -0.005104623596081216
    seekback_1to2 = 0.0014180413332786782
    seekback_3+ = 0.014109003958404538
    jan = -0.00872984593168581
    feb = -3.032663944623929e-06
    march = 0.0269911027449358
    april = 0.035469448420248774
    may = 0.04152334798981754
    june = 2.8242489632087233e-05
    july = -0.025102079856237466
    aug = 0.028206070469311815
    sept = 0.026958566561161995
    nov = -0.008621813089357886
    afternoon = -0.00582052079141175
    early_morn = -0.0015258072940813228
    morning = 0.009064352472990256
    night = -0.00011911576207420039
    skip_v_soon = -0.019312944098743023
    skip_fairly_soon = 0.01503217220739842
    switched_context = 0.008349517904256558
    paused_before_play = -0.008278713572397452
    on_shuffle = -0.007133054510422039
    premium = -0.008370161411441814
    context_catalog = -0.00

    context_radio = 0.019467141906669462
    context_user_collec = -0.016320985696168192
    appload = 0.025028906492917707
    start_backbutton = 0.019787953951635042
    start_clickrow = -0.028948040908078534
    start_endplay = -0.007564685086557398
    start_fwdbutton = -0.0029021881580022293
    start_playbutton = -0.07649217472029012
    start_remote = 0.03185581062435563
    start_trackdone = 0.006226515862318551
    start_uriopen = 0.02374403075685717
    skipped = 0.020835534520494213
For cluster 11: 
    no_seekfwd = -0.00909008380182723
    seekfwd_1to2 = 0.0040135994450453275
    seekfwd_3+ = 0.017005524672310916
    no_seekback = -0.012499678785714043
    seekback_1to2 = 0.007588762672370598
    seekback_3+ = 0.019533552382522218
    jan = -0.0021620503535942125
    feb = 1.75534933383388e-07
    march = -0.001845668800217498
    april = -0.002011275160753275
    may = -0.01387681818996663
    june = 0.005341421043792467
    july = 0.0009334741300585288
    aug = -0.005444

In [819]:
#  I want to see the maximum coefs for each cluster along with corresponding column names:
for cluster in range(len(cluster_clf2_fit.coef_)):
    max_seen = float("-inf")
    column_name = None
    for index, coef in enumerate(cluster_clf2_fit.coef_[cluster]):
        if coef <= max_seen: continue 
        column_name = Xclusters_track10_dummies.columns[index]
        max_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), max_seen, column_name)

Cluster -1 Max Coef:  0.04152334798981754 may
Cluster 0 Max Coef:  0.07327770574337121 july
Cluster 1 Max Coef:  0.05459843899277006 may
Cluster 2 Max Coef:  0.06117623630232805 start_remote
Cluster 3 Max Coef:  0.05411762779486266 skip_fairly_soon
Cluster 4 Max Coef:  0.07500886597112591 july
Cluster 5 Max Coef:  0.04331642449996252 skip_fairly_soon
Cluster 6 Max Coef:  0.04486231779392105 start_remote
Cluster 7 Max Coef:  0.06372559321644178 start_clickrow
Cluster 8 Max Coef:  0.054218775590222215 may
Cluster 9 Max Coef:  0.06850894548576854 skip_v_soon
Cluster 10 Max Coef:  0.03896596614791819 aug
Cluster 11 Max Coef:  0.05156906354394538 start_remote
Cluster 12 Max Coef:  0.05757453984422139 switched_context


In [ ]:
# Print out absolute value maximum coefficients (whether pos or neg)
for cluster in range(len(cluster_clf2_fit.coef_)):
    max_seen = 0
    column_name = None
    for index, coef in enumerate(cluster_clf2_fit.coef_[cluster]):
        abs_coef = abs(coef)
        if abs_coef <= abs(max_seen): continue 
        column_name = Xclusters_track10_dummies.columns[index]
        max_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), max_seen, column_name)

In [ ]:
# Print out minimum coefficients (which in this case actually means they are the strongest negative ones)
for cluster in range(len(cluster_clf2_fit.coef_)):
    min_seen = float("inf")
    column_name = None
    for index, coef in enumerate(cluster_clf2_fit.coef_[cluster]):
        if coef >= min_seen: continue 
        column_name = Xclusters_track10_dummies.columns[index]
        min_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), min_seen, column_name)

In [ ]:
# Now we'll get the odds ratios for track10

t10_odds = np.exp(cluster_clf2_fit.coef_)
t10_odds

In [ ]:
# I want to print out the maximum odds ratios for each column:
for cluster in range(len(t10_odds)):
    max_seen = float("-inf")
    column_name = None
    for index, oratio in enumerate(t10_odds[cluster]):
        if oratio <= max_seen: continue 
        column_name = Xclusters_track10_dummies.columns[index]
        max_seen = oratio
    print('Cluster {} Max Odds Ratio: '.format(cluster-1), max_seen, column_name)

In [ ]:
# Now we'll run a logistic regression on the clusters for track20
scaler = StandardScaler()
X20 = scaler.fit_transform(Xclusters_track20_dummies)
y = track20_hd_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=60)

In [ ]:
cluster_clf3 = LogisticRegression(multi_class ='multinomial', solver ='newton-cg')
cluster_clf3_fit = cluster_clf.fit(X_train, y_train)

In [ ]:
y_pred = cluster_clf3_fit.predict(X_test)

In [ ]:
# Now a confusion matrix for the track1 clusters' logistic regression
cm3 = confusion_matrix(y_test, y_pred)
print(cm3)

In [ ]:
print(cluster_clf3_fit.coef_)
print(cluster_clf3_fit.classes_)

In [ ]:
#Okay, let's look at these coeffs in an easier to interpret format

for cluster in range(len(cluster_clf3_fit.coef_)):
    print('For cluster {}: '.format(cluster-1))
    for coef in range(len(cluster_clf3_fit.coef_[cluster])):
        print('   ', Xclusters_track20_dummies.columns[coef], '=', cluster_clf3_fit.coef_[cluster][coef])

In [ ]:
# I want to print out the actual column names for the max coefficients :
for cluster in range(len(cluster_clf3_fit.coef_)):
    max_seen = float("-inf")
    column_name = None
    for index, coef in enumerate(cluster_clf3_fit.coef_[cluster]):
        if coef <= max_seen: continue 
        column_name = Xclusters_track20_dummies.columns[index]
        max_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), max_seen, column_name)

In [ ]:
# Print out absolute value maximum coefficients (whether pos or neg)
for cluster in range(len(cluster_clf3_fit.coef_)):
    max_seen = 0
    column_name = None
    for index, coef in enumerate(cluster_clf3_fit.coef_[cluster]):
        abs_coef = abs(coef)
        if abs_coef <= abs(max_seen): continue 
        column_name = Xclusters_track20_dummies.columns[index]
        max_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), max_seen, column_name)

In [ ]:
# Print out minimum coefficients (which in this case actually means they are the strongest negative ones)
for cluster in range(len(cluster_clf3_fit.coef_)):
    min_seen = float("inf")
    column_name = None
    for index, coef in enumerate(cluster_clf3_fit.coef_[cluster]):
        if coef >= min_seen: continue 
        column_name = Xclusters_track20_dummies.columns[index]
        min_seen = coef
    print('Cluster {} Max Coef: '.format(cluster-1), min_seen, column_name)

In [ ]:
# Now get odds ratios for track20

t20_odds = np.exp(cluster_clf2_fit.coef_)
t20_odds

In [ ]:
# I want to print out the maximum odds ratios for each column:
for cluster in range(len(t20_odds)):
    max_seen = float("-inf")
    column_name = None
    for index, oratio in enumerate(t20_odds[cluster]):
        if oratio <= max_seen: continue 
        column_name = Xclusters_track20_dummies.columns[index]
        max_seen = oratio
    print('Cluster {} Max Odds Ratio: '.format(cluster-1), max_seen, column_name)

In [ ]:
# First checking length of cluster labels array is same length as track1 df
print(len(track1_hd_labels))

In [ ]:
# Placing each observation's cluster assignment in the df 
Xclusters_track1_dummies['hd_clusters'] = pd.Series(track1_hd_labels, index=Xclusters_track1_dummies.index)

In [ ]:
# See the cluster breakdown
print(Xclusters_track1_dummies['hd_clusters'].unique())
print(Xclusters_track1_dummies['hd_clusters'].value_counts())

Now we'll add the clusters as an array back into the track1 df so we can investigate them further

Next up, let's do some barplots with the clusters to investigate a bit more what they mean.

I'll choose 6 variables to look at with the barplots. I can always do more later if I need to.

1. premium
2. skipped
3. reason_start_num
4. on_shuffle
5. context
6. hour_of_day

In [ ]:
# Plot clusters with premium
sns.catplot(x='hd_clusters', y='premium', kind='bar', palette='Greens_r', data=Xclusters_track1_dummies)
plt.title('Relationship between Clusters & Premium Membership')
plt.xlabel('Clusters')
plt.ylabel('Premium Membership')
plt.gcf().set_size_inches(15,10)
plt.show()

In [ ]:
# Plot clusters with skipped
sns.catplot(x='hd_clusters', y='skipped', kind='bar', palette='Greens_r', data=Xclusters_track1_dummies)
plt.title('Relationship between Clusters & Skipped')
plt.xlabel('Clusters')
plt.ylabel('Skipped')
plt.gcf().set_size_inches(15,10)
plt.show()

In [ ]:
# Let's see what a non-binary dummy var looks like
sns.catplot(x='hd_clusters', y='night', kind='bar', palette='Greens_r', data=Xclusters_track1_dummies)
plt.title('Relationship between Clusters & Night-time Listening')
plt.xlabel('Clusters')
plt.ylabel('Night-time Listening')
plt.gcf().set_size_inches(15,10)
plt.show()

That's interesting. It looks like Cluster 11 had many more incidences of night-time listening sessions than the others. I will keep the dataframe the dummies dataframe then, rather than using the df that existed pre-one-hot encoding. Looking specifically at each dummy value seems like it may be more illuminating.

In [ ]:
# Let's look at all of the vars in the dummies here, and then later we can choose which will go in the presentation
for col in Xclusters_track1_dummies:
    sns.catplot(x='hd_clusters', y=col, kind='bar', palette='Greens_r', data=Xclusters_track1_dummies)
    plt.title('Relationship between Clusters & {}'.format(col))
    plt.xlabel('Clusters')
    plt.ylabel('{}'.format(col))
    plt.gcf().set_size_inches(15,10)
    plt.show()

Now we'll add the clusters as an array back into the track10 df so we can investigate them further

In [ ]:
# First checking length of cluster labels array is same length as track10 df
print(len(track10_hd_labels))

In [ ]:
# Placing each observation's cluster assignment in the df 
Xclusters_track10_dummies['hd_clusters'] = pd.Series(track10_hd_labels, index=Xclusters_track10_dummies.index)

In [ ]:
# Look at the new column; make sure it's okay
Xclusters_track10_dummies.head(20)

In [ ]:
# See the cluster breakdown
print(Xclusters_track10_dummies['hd_clusters'].unique())
print(Xclusters_track10_dummies['hd_clusters'].value_counts())

Next up, let's do some barplots with the clusters to investigate a bit more what they mean.

In [ ]:
# Let's look at all of the vars in the dummies here, and later we can choose which will go in the presentation
for col in Xclusters_track10_dummies:
    sns.catplot(x='hd_clusters', y=col, kind='bar', palette='Greens_r', data=Xclusters_track10_dummies)
    plt.title('Relationship between Clusters & {}'.format(col))
    plt.xlabel('Clusters')
    plt.ylabel('{}'.format(col))
    plt.gcf().set_size_inches(15,10)
    plt.show()

Now we'll add the clusters as an array back into the track20 df so we can investigate them further

In [ ]:
# First checking length of cluster labels array is same length as track10 df
print(len(track20_hd_labels))

In [ ]:
# Placing each observation's cluster assignment in the df 
Xclusters_track20_dummies['hd_clusters'] = pd.Series(track20_hd_labels, index=Xclusters_track20_dummies.index)

In [ ]:
# Look at the new column; make sure it's okay
Xclusters_track20_dummies.head(20)

In [ ]:
# See the cluster breakdown
print(Xclusters_track20_dummies['hd_clusters'].unique())
print(Xclusters_track20_dummies['hd_clusters'].value_counts())

Next up, let's do some barplots with the clusters to investigate a bit more what they mean.

In [ ]:
# Let's look at all of the vars in the dummies here, and later we can choose which will go in the presentation
for col in Xclusters_track20_dummies:
    sns.catplot(x='hd_clusters', y=col, kind='bar', palette='Greens_r', data=Xclusters_track20_dummies)
    plt.title('Relationship between Clusters & {}'.format(col))
    plt.xlabel('Clusters')
    plt.ylabel('{}'.format(col))
    plt.gcf().set_size_inches(15,10)
    plt.show()

In [ ]:
# metrics: AUC score;
# for regression: loss function; F1 score
# use train val & test set: parfit

In [ ]:
#CREATE THE SEPARATED ANALYSIS SPECIFIC DATASETS SO THAT THE OUTCOME VARIABLE IS ALONE AND READY FOR THE ANALYSIS
# track1_sl = Xclusters_track1_dummies.drop(columns='skipped')
# X, y = track1_sl.iloc[:,:],Xclusters_track1_dummies.iloc[:,41]

In [ ]:
# Split into train, val, & test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=968)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=987)

In [ ]:
# grid1 = {
#     'max_iter': [50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 200, 500, 700, 900, 1100],
#     'n_jobs': [-1],
#     'random_state': [4218]
# }
# paramGrid = ParameterGrid(grid2)
# best_model, best_score, all_models, all_scores = bestFit(LogisticRegression(),paramGrid,
#                                                     y_train, y_train, X_val, X_val,
#                                                     metric=log_loss, greater_is_better=False,
#                                                     scoreLabel='log_loss')
# print(best_model, best_score)

In [779]:
# pipe1 = Pipeline([('scaler', scaler), ('regression', cluster_clf1_fit)])

# pipe_pred = pipe1.predict(X1_test)
# pipe_pred

array([ 1, -1,  8, ...,  0,  3,  8])